# <a id='model-selection-and-evaluation'></a> 3. [**Sélection et évaluation de modèle**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#model-selection-and-evaluation)</br>([*Model selection and evaluation*](https://scikit-learn.org/stable/model_selection.html#model-selection-and-evaluation))

# 3.1. [**Validation croisée : évaluer les performances des estimateurs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-evaluating-estimator-performance)<br/>([_Cross-validation: evaluating estimator performance_](https://scikit-learn.org/stable/model_selection.html#cross-validation-evaluating-estimator-performance))

# Sommaire

- **Volume** : 25 pages, 7 exemples, 7 papiers
- 3.1.1. [**Calcul de métriques à validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#computing-cross-validated-metrics)<br/>([_computing-cross-validated-metrics_](https://scikit-learn.org/stable/model_selection.html#computing-cross-validated-metrics))
- 3.1.2. [**Itérateurs de validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-iterators)<br/>([_Cross validation iterators_](https://scikit-learn.org/stable/model_selection.html#cross-validation-iterators))
- 3.1.3. [**Une note sur le brassage**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#a-note-on-shuffling)<br/>([_A note on shuffling_](https://scikit-learn.org/stable/model_selection.html#a-note-on-shuffling))
- 3.1.4. [**Validation croisée et sélection de modèles**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-and-model-selection)<br/>([_cross-validation-and-model-selection_](https://scikit-learn.org/stable/model_selection.html#cross-validation-and-model-selection))
- 3.1.5. [**Résultat du test de permutation**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#permutation-test-score)<br/>([_Permutation test score_](https://scikit-learn.org/stable/model_selection.html#permutation-test-score))